In [1]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
data = pd.read_csv(r'C:\Users\Ted\Desktop\Juju\DEV\credit-risk-analysis\data\data.csv')

In [3]:
# 1. Create Aggregate Features
# Create aggregate features for each customer
aggregate_features = data.groupby('CustomerId').agg(
    TotalTransactionAmount=('Amount', 'sum'),
    AverageTransactionAmount=('Amount', 'mean'),
    TransactionCount=('TransactionId', 'count'),
    StdDevTransactionAmount=('Amount', 'std')
).reset_index()

# Display the first few rows of the aggregated features
aggregate_features.head()


,CustomerId,TotalTransactionAmount,AverageTransactionAmount,TransactionCount,StdDevTransactionAmount
0,CustomerId_1,-10000.0,-10000.000000,1,NaN
1,CustomerId_10,-10000.0,-10000.000000,1,NaN
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333
3,CustomerId_1002,4225.0,384.090909,11,560.498966
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146


In [4]:
# 2. Extract Features
# Extract new features from TransactionStartTime
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])
data['TransactionHour'] = data['TransactionStartTime'].dt.hour
data['TransactionDay'] = data['TransactionStartTime'].dt.day
data['TransactionMonth'] = data['TransactionStartTime'].dt.month
data['TransactionYear'] = data['TransactionStartTime'].dt.year

# Display the first few rows of the dataset with new features
data.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionHour,TransactionDay,TransactionMonth,TransactionYear
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,2018


In [5]:
# 3. Encode Categorical Variables
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# One-Hot Encoding for categorical variables
categorical_columns = ['CurrencyCode', 'ProviderId', 'ProductCategory', 'ChannelId']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Label Encoding for categorical variables
label_encoder = LabelEncoder()
label_encoded_columns = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProductId']
for col in label_encoded_columns:
    data[col] = label_encoder.fit_transform(data[col])

# Display the first few rows of the dataset with encoded features
data.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProductId,Amount,Value,TransactionStartTime,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,78150,46980,2490,3535,2584,256,1,1000.0,1000,2018-11-15 02:18:49+00:00,...,False,False,False,False,False,False,False,False,True,False
1,75821,31755,3219,2366,2584,256,19,-20.0,20,2018-11-15 02:19:08+00:00,...,True,False,False,False,False,False,False,True,False,False
2,39888,60272,2713,996,2806,256,0,500.0,500,2018-11-15 02:44:21+00:00,...,False,False,False,False,False,False,False,False,True,False
3,48738,1797,3351,974,3733,256,11,20000.0,21800,2018-11-15 03:32:55+00:00,...,False,False,False,False,False,False,True,False,True,False
4,41364,48941,3219,2366,3733,256,19,-644.0,644,2018-11-15 03:34:21+00:00,...,True,False,False,False,False,False,False,True,False,False


In [10]:
# 4. Handle Missing Values
# Handling missing values using imputation 
data.fillna(data.mean(), inplace=True)


In [11]:
# 5. Normalize/Standardize Numerical Features
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Select numerical columns for normalization/standardization
numerical_columns = ['Amount', 'Value', 'TransactionHour', 'TransactionDay', 'TransactionMonth', 'TransactionYear']

# Normalize numerical features
scaler = MinMaxScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Standardize numerical features
standardizer = StandardScaler()
data[numerical_columns] = standardizer.fit_transform(data[numerical_columns])

# Display the first few rows of the dataset with scaled features
data.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProductId,Amount,Value,TransactionStartTime,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,78150,46980,2490,3535,2584,256,1,-0.046371,-0.072291,2018-11-15 02:18:49+00:00,...,False,False,False,False,False,False,False,False,True,False
1,75821,31755,3219,2366,2584,256,19,-0.054643,-0.080251,2018-11-15 02:19:08+00:00,...,True,False,False,False,False,False,False,True,False,False
2,39888,60272,2713,996,2806,256,0,-0.050426,-0.076352,2018-11-15 02:44:21+00:00,...,False,False,False,False,False,False,False,False,True,False
3,48738,1797,3351,974,3733,256,11,0.107717,0.096648,2018-11-15 03:32:55+00:00,...,False,False,False,False,False,False,True,False,True,False
4,41364,48941,3219,2366,3733,256,19,-0.059704,-0.075183,2018-11-15 03:34:21+00:00,...,True,False,False,False,False,False,False,True,False,False
